Import Stuff

In [4]:
import ssl
import re

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP

from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob

import pickle
import requests





In [16]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'
df_manifesto_toks = pd.read_csv("data/parties/Manifestos_tokenized.csv")
df_manifesto = pd.read_csv("data/parties/All_Manifestos.csv")
topics = pd.read_pickle('topics.pkl') 


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


DeepL API

In [6]:
params = {
    'auth_key': api_key_deepl,
    'text': 'Hello, world!',
    'target_lang': 'DE'  # Translate to German
}

In [7]:
# Make the API request
response = requests.post(url, data=params)


In [8]:
# Check if the request was successful
if response.status_code == 200:
    result = response.json()
    translated_text = result['translations'][0]['text']
    print(f'Translated text: {translated_text}')
else:
    print(f'Error: {response.status_code}')
    print(response.text)

Translated text: Hallo, Welt!


Edit Text documents

In [9]:
lemmatizer = WordNetLemmatizer()

defining different models
    

In [8]:
umap_model = UMAP(
    n_neighbors=50, 
    # n_components=5, 
    # min_dist=0.0, 
    metric='cosine', 
    random_state=42
    )

In [9]:
embedding_model_en = "all-MiniLM-L6-v2"
embedding_model_multilingual1 = "paraphrase-multilingual-mpnet-base-v2"
embedding_model_multilingual2 = "distiluse-base-multilingual-cased-v1"

In [10]:
representation_model = KeyBERTInspired()
cluster_model = KMeans(n_clusters=30)

In [11]:
hdbscan_model = HDBSCAN(
    min_cluster_size=50, 
    # min_samples=10,
    metric='euclidean', 
    prediction_data=True
    )



In [12]:
seed_topic_list = [["steuergeld", "ausgaben", "staatsausgaben", "staatshaushalt", "schulden", "schuldenbremse", "staatsschulden"],
                   ['sozial wohnungsbau', 'sozialen wohnungsbaus', 'sozialen wohnungsbau', 'sozialwohnungen', 'wohnungsbauförderung', 'mietwohnungen', 'wohnungsbaus', 'bezahlbar wohnungen', 'eigentumswohnungen', 'wohnungsbau'],
                   ['öffentlich investitionen', 'öffentlichen investitionen', 'investitionen öffentlichen', 'investitionen', 'investitionsquot', 'investitionsstau', 'investitionszulag', 'investitionsmittel', 'privat investitionen', 'investitionsanr']]

Initialisiere und trainiere das BERTopic-Modell


In [17]:

# Ensure the 'text' column contains only strings and handle NaN values
df_manifesto["text"] = df_manifesto["text"].astype(str).fillna("")

topic_model = BERTopic(
    language="multilingual"
    , n_gram_range=(1,2)
    , min_topic_size=20
    # , top_n_words=20
    , representation_model=representation_model
    , embedding_model=embedding_model_multilingual2
    , umap_model=umap_model
    # , hdbscan_model=hdbscan_model
    , seed_topic_list=seed_topic_list
    )

#topic_model = BERTopic(nr_topics=100, calculate_probabilities=True, embedding_model="all_MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(df_manifesto["text"])



In [17]:
df_manifesto["topic"] = topics

In [18]:
df_manifesto.to_pickle('Manifesto_final.pkl')

In [32]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,38779,-1_sicherheit_demokratisch_demokrati_sozial,"[sicherheit, demokratisch, demokrati, sozial, ...",[müssen entscheiden mutig anpacken sozial ökol...
1,0,8594,0_kindern_kinderbetreuung_kindergeld_familien,"[kindern, kinderbetreuung, kindergeld, familie...",[Kinder brauchen Kinder umso mehr angesicht Tr...
2,1,3254,1_arbeitsbedingungen_arbeitnehmerinnen arbeitn...,"[arbeitsbedingungen, arbeitnehmerinnen arbeitn...",[Arbeit Prei gute Arbeitsbedingungen Arbeit ge...
3,2,2026,2_erneuerbaren energien_erneuerbar energien_au...,"[erneuerbaren energien, erneuerbar energien, a...",[Unterstützung Ausbau erneuerbar Energien sowi...
4,3,1986,3_sozial wohnungsbau_sozialen wohnungsbaus_soz...,"[sozial wohnungsbau, sozialen wohnungsbaus, so...",[Ausreichend Wohnungen bezahlbaren Preisen ger...
...,...,...,...,...,...
209,208,21,208_verbraucherschutzverbänd sollen_fördert ve...,"[verbraucherschutzverbänd sollen, fördert verb...",[Verbraucherschutzverbänd sollen finanziel bes...
210,209,21,209_arbeitsmarkt steuerreform_steuerreform ent...,"[arbeitsmarkt steuerreform, steuerreform entbü...",[Tourismus wichtig Wirtschaftsfaktor Klein mit...
211,210,21,210_verbesserung umweltpolitik_umweltpolitik s...,"[verbesserung umweltpolitik, umweltpolitik stä...",[unser Aufgab natürlichen Lebensbedingungen mi...
212,211,21,211_hoheitsverwaltung muß_möchten gesellschaft...,"[hoheitsverwaltung muß, möchten gesellschaftli...",[GRÜNE möchten gesellschaftlichen Zwang Immer ...


In [ ]:
len(topic_model.get_topic(25))

In [ ]:

# Erstelle das `topics_over_time`-Objekt
topics_over_time = topic_model.topics_over_time(df_manifesto["text"], df_manifesto["date"])
#topic_model = BERTopic(representation_model=KeyBERTInspired())


In [ ]:
# Save topics_over_time to a file
with open("topics_over_time.pkl", "wb") as f:
    pickle.dump(topics_over_time, f)

In [38]:

# Method 1 - safetensors
topic_model.save(file_path, serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model_multilingual2)

# Method 2 - pytorch
#topic_model.save(file_path, serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 3 - pickle
#topic_model.save("bertopic_model.pkl")



In [39]:
# Save the topics to a pickle file
with open('topics.pkl', 'wb') as f:
	pickle.dump(topics, f)

# Save the probabilities to a pickle file
with open('probs.pkl', 'wb') as f:
	pickle.dump(probs, f)